# ルートを評価するために解くLPについて
- ルートが1つ与えられた後、そのルートを評価するためにLPを解く(ルートが実行可能であるとは限らない)

<!--- 各制約の違反度を変数とし、それらに重みをかけて足し合わせた関数の最小化問題とする-->
- 車両が各顧客へ到着する時刻を変数とし、その合計を最小化する問題とする
- 制約は、容量制約と時間枠制約とする
    - 容量制約は、ある区間における2つの関数の積分値(面積)の大小を比較するという形で表す(車両の積荷の量を表す区分線形関数の\[x0, xn\]までの積分値と最大容量を表す線形関数の\[x0, xn\]までの積分値)
    - 時間枠制約は、通常のVRPの定式化と同様に表す


# 前準備

## 問題例の読み込み
使用した問題例 : https://neo.lcc.uma.es/vrp/vrp-instances/description-for-files-of-solomons-instances/

In [ ]:
class Customer():
    def __init__(self, x, y, d, e, l, s):
        self.x = x
        self.y = y
        self.d = d
        self.e = e
        self.l = l
        self.s = s

In [ ]:
import glob

files = glob.glob("./solomon_25/*")
for file in files:
    print(file)

In [ ]:
# 問題例の選択
instance = files[0]

In [ ]:
Customers = {}
with open(instance, mode="r") as f:
    for index,line in enumerate(f):
        #print(line)
        l = []
        if index==4:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            K, Q = l
        elif 9<=index<=34:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            Customers[l[0]] = Customer(*l[1:])

In [ ]:
Customers["depot"] = Customer(0,0,0,0,1000,0)

In [ ]:
print("\t", list(vars(Customers["depot"]).keys()))
for i in Customers.keys():
    print(i, end="\t")
    for key, val in vars(Customers[i]).items():
        print(val, end=" ")
    print()

## ルートの作成

In [ ]:
# 入力
## 顧客
C={} # 客の座標を保存する辞書
TW={} # 客の時間枠を保存する辞書
demand={} # 客の要求量(正の値は集荷，負の値は配達)
S={} # 客のサービス時間

for i in Customers:
    x, y = Customers[i].x, Customers[i].y
    C[i] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[i] = (e,l)
    demand[i] = (Customers[i].d, )
    S[i] = Customers[i].s
    """name="c"+str(i)
    x, y = Customers[i].x, Customers[i].y
    C[name] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[name] = (e,l)
    demand[name] = (Customers[i].d, )
    S[name] = Customers[i].s"""
#C["depot"]=(0,0)

## 車両
M = K//2 # number of vehicles
capacities = [(Q*2, ) for k in range(M)] # capacity of vehicle

In [ ]:
print("Customer",C)
print("Time Window",TW)
print("Demand",demand)
print("Service Time",S)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(7,7))

G=nx.DiGraph()
nx.draw_networkx(G,pos=C,nodelist=[i for i in C if i != "depot"],node_color="y",node_size=50,with_labels=True,edge_color="k",width=1)
nx.draw_networkx(G,pos=C,nodelist=["depot"],node_color="blue",node_shape='s',alpha=0.5,node_size=100,with_labels=True,edge_color="k",width=1)

plt.show()

In [ ]:
# 距離関数の定義
#def Distance(t1,t2):
#    return ((t1[0]-t2[0])**2+(t1[1]-t2[1])**2)**(0.5)
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

# ソルバーの読み込み
import sys
sys.path.append('..')

#from vrplib.vrp_d_1m1_t_model import *
import vrplib.vrp_d_1m1_t_model as vrp

# ソルバーの実行
model = vrp.Model(file.split("/")[-1].split(".")[0]) # モデルインスタンスの生成

## 客インスタンスの生成
for i in C:
    if i == "depot":
        continue
    model += vrp.Customer(i,demand=demand[i],timewindow=TW[i],servicetime=S[i])

## 車両インスタンスの生成
for k in range(M):
    model += vrp.Vehicle("v"+str(k),capacity=capacities[k])
    
## 枝インスタンスの生成
for i in C:
    for j in C:
        if i!=j:
            dist = time = distance(*C[i],*C[j])
            model += vrp.Edge(i,j,dist,time)

In [ ]:
## 最適化の実行
obj=model.optimize(IterLimit=10000,TimeLimit=10,Verbose=False,OutputFlag=False)

## 得られた解の表示
print("objective value =",obj)
for v in model.vehiclesL:
    print()
    print(v)
    print(list(map(lambda x:x.name,v.routing[1:-1])))    
    for iv in map(lambda x:x.name,v.routing[1:-1]):
        print(iv)

## ルートを1つ選ぶ

In [ ]:
# Vehicle 6の巡回路を選んでいる
tour = [12, 3, 24, 25]#+["depot"]

## その他作業

In [ ]:
# 使用した問題例の名称を保存
tmp = []
for s in instance[-1:0:-1]:
    if s=='/':
        break
    tmp.append(s)
instance_name = "".join(tmp[-1:0:-1])
print(instance_name)

# 前準備

## instances.pyからインスタンスを得る

In [1]:
import instances
Customers = instances.Customers
tour = instances.tour

In [2]:
instance_name = "C" + str(len(Customers)-1)

In [3]:
tour[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
tour[-10:]

[865, 866, 867, 868, 869, 870, 871, 872, 873, 'depot']

# 問題を解く
2つの手法に対し、計算時間を比較

## 入力する行列、ベクトルの作成

In [5]:
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

In [6]:
def make_preceding_constr(tour, Customers, Ax, Ap, b, c):
    # ルート内の顧客の順序に関する制約
    for index, i in enumerate(tour):
        try:
            i_next = tour[index+1]
        except:
            i_next = "depot"
            continue
        Ax.append([1 if target==i else -1 if target==i_next else 0 for target in tour])
        Ap.append([0 for _ in range(len(tour))])
        b.append(-distance(Customers[i].x, Customers[i].y, Customers[i_next].x, Customers[i_next].y))
    return Ax, Ap, b, c

In [7]:
def make_tw_constr(tour, Customers, Ax, Ap, b, c):
    # 時間枠制約
    for index, i in enumerate(tour):
        Ax.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(-Customers[i].e)
        Ax.append([1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(Customers[i].l)
    return Ax, Ap, b, c

In [8]:
def make_inputs(tour, Customers):
    import numpy as np
    Ax, Ap, b, c = [], [], [], [1 for _ in range(len(tour))]
    # ルート内の顧客の順序に関する制約
    Ax, Ap, b, c = make_preceding_constr(tour, Customers, Ax, Ap, b, c)
    print("Preceding constraints are done.")
    # 時間枠制約
    Ax, Ap, b, c = make_tw_constr(tour, Customers, Ax, Ap, b, c)
    print("Time-window constarints are done.")
    # ndarrayに変換
    Ax = np.array(Ax)
    Ap = np.array(Ap)
    b = np.array(b)
    c = np.array(c)
    return Ax, Ap, b, c

## 主問題を解く関数の定義
与えられた定数を元にLPのモデルを作成した上でそれを解き、最適解を返す関数

In [14]:
def solve_primal(Ax, Ap, b, c, instance_name, num_vars):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    # 定数を設定　←　入力として与えられる
    # 変数を設定
    """
    x_i : 顧客iへ車両が到着する時刻を表す変数
    p_i : 顧客iの時間枠の違反度を表す変数
    """
    x = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="x")
    p = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="p")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(c.T @ p, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax @ x + Ap @ p <= b, name="c")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    """if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"車両が顧客{i}に到着する時刻は、{x[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')"""
        
    return m

## 双対問題を解く関数の定義

In [64]:
def solve_dual(Ax, Ap, b, c, instance_name, num_vars, PStarts, DStarts):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    
    # 変数を設定
    """
    y_i : 主問題における制約式iの潜在価値
    """
    y = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="y")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(-1 * b.T @ y, sense=gp.GRB.MAXIMIZE)
    
    # 制約条件を設定
    c1 = m.addConstr(Ax.T @ y >= 0, name="c1")
    c2 = m.addConstr(Ap.T @ y + c >= 0, name="c2")
    c3 = m.addConstr(y >= 0, name="c3")
    c = c1+ c2 + c3

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # ホットスタートの使用
    for i in range(num_vars):
        y[i].PStart = PStarts[i]
    for i in range(len(c)):
        c[i].DStart = DStarts[i]
    
    # 最適化
    m.Params.Method = 0
    #m.Params.Presolve = 0
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    """if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"主問題における制約{i}の潜在価格は、{y[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')"""
        
    return m

## ①全体を1つのLPとして解くveb.

### 全体のPrimalを解く

In [65]:
Ax, Ap, b, c = make_inputs(tour, Customers)

Preceding constraints are done.
Time-window constarints are done.


In [66]:
# Gurobiによって最適解を求める
P = solve_primal(Ax, Ap, b, c, instance_name+"P", len(tour))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2624 rows, 1750 columns and 5248 nonzeros
Model fingerprint: 0xa52d3344
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 1940 rows and 1069 columns
Presolve time: 0.01s
Presolved: 684 rows, 681 columns, 1368 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.9048706e+05   5.161921e+00   0.000000e+00      0s
     679    8.9132657e+05   0.000000e+00   0.000000e+00      0s

Solved in 679 iterations and 0.04 seconds
Optimal objective  8.913265696e+05


In [ ]:
for var in P.getVars():
    print(var.varName, var.X)

In [ ]:
for constr in P.getConstrs():
    print(constr.Pi)

## ②前半と後半をつなげるveb.
1. 適当なところで前後に分ける
1. 前半後半それぞれのPrimalを解く
1. 前半と後半それぞれのPrimalの最適解を、全体のDualに入れて解く
1. 全体のPrimalの最適解を得る

### 1. 適当なところで前後に分ける
- ひとまず半分くらいで分けることにする

In [18]:
threshold = len(tour)//2
#print(threshold)

former = tour[:threshold]
latter = tour[threshold:]
#print(f"巡回路全体は、{tour}")
print(f"前半は、{former}")
print(f"後半は、{latter}")

前半は、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 

### 2. 前半後半それぞれのPrimalを解く

In [19]:
import numpy as np
# 係数行列、ベクトルを整える
A1x, A1p, A2x, A2p, A3x, A3p, b1, b2, b3, c1, c2 = [], [], [], [], [], [], [], [], [], [], []
## A1, A2, A3を定める
## b1, b2, b3を定める
for Ax_i, Ap_i, b_i in zip(Ax, Ap, b):
    if np.linalg.norm(Ax_i[threshold:], ord=2)==0.:
        A1x.append(Ax_i[:threshold])
        A1p.append(Ap_i[:threshold])
        b1.append(b_i)
    elif np.linalg.norm(Ax_i[:threshold], ord=2)==0.:
        A2x.append(Ax_i[threshold:])
        A2p.append(Ap_i[threshold:])
        b2.append(b_i)
    else:
        A3x.append(Ax_i)
        A3p.append(Ap_i)
        b3.append(b_i)
## c1, c2, c3を定める
c1 = c[:threshold]
c2 = c[threshold:]
## リストからnumpy arrayに変換
A1x = np.array(A1x)
A1p = np.array(A1p)
A2x = np.array(A2x)
A2p = np.array(A2p)
A3x = np.array(A3x)
A3p = np.array(A3p)
b1 = np.array(b1)
b2 = np.array(b2)
b3 = np.array(b3)
c1 = np.array(c1)
c2 = np.array(c2)
for i in range(1, 4):
    print(f"A{i}x : ", end="")
    print(eval("A"+str(i)+"x"))
    print(f"A{i}p : ", end="")
    print(eval("A"+str(i)+"p"))
    print(f"b{i} : ", end="")
    print(eval("b"+str(i)))
    if i <= 2:
        print(eval("c"+str(i)))

A1x : [[ 1 -1  0 ...  0  0  0]
 [ 0  1 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0  1]]
A1p : [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0 -1]]
b1 : [   -2.23606798    -2.23606798    -3.16227766 ...  1515.
 -1515.          1515.        ]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [20]:
# Gurobiによって最適解を求める
P_f = solve_primal(A1x, A1p, b1, c1, instance_name+"P_f", len(former))
P_l = solve_primal(A2x, A2p, b2, c2, instance_name+"P_l", len(latter))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1310 rows, 874 columns and 2620 nonzeros
Model fingerprint: 0x37b31d00
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 1073 rows and 640 columns
Presolve time: 0.03s
Presolved: 237 rows, 234 columns, 474 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4135279e+05   4.866284e+00   0.000000e+00      0s
     232    2.4154259e+05   0.000000e+00   0.000000e+00      0s

Solved in 232 iterations and 0.03 seconds
Optimal objective  2.415425934e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1313 rows, 876 columns and 2626 nonzeros
Model fingerprint: 0xfddb86b5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+03]
Pr

In [21]:
for v in P_f.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

x[0] 0 -1
x[1] 2.23607 0
x[2] 4.47214 0
x[3] 7.63441 0
x[4] 17.5339 0
x[5] 28.1641 0
x[6] 39.4778 0
x[7] 50.8795 0
x[8] 59.4235 0
x[9] 63.0291 0
x[10] 71.0913 0
x[11] 77.7995 0
x[12] 88.6162 0
x[13] 94.4471 0
x[14] 98.4471 0
x[15] 100.683 0
x[16] 102.919 0
x[17] 108.919 0
x[18] 111.919 0
x[19] 116.391 0
x[20] 117.806 0
x[21] 119.806 0
x[22] 126.13 0
x[23] 130.253 0
x[24] 137.533 0
x[25] 145.344 0
x[26] 156.16 0
x[27] 164.223 0
x[28] 172.033 0
x[29] 180.518 0
x[30] 186.601 0
x[31] 196.896 0
x[32] 204.108 0
x[33] 211.319 0
x[34] 213.555 0
x[35] 214.969 0
x[36] 220.068 0
x[37] 223.23 0
x[38] 233.526 0
x[39] 244.84 0
x[40] 256.881 0
x[41] 265.825 0
x[42] 275.259 0
x[43] 285.89 0
x[44] 289.052 0
x[45] 295.135 0
x[46] 302.945 0
x[47] 306.55 0
x[48] 311.55 0
x[49] 322.952 0
x[50] 329.66 0
x[51] 336.064 0
x[52] 342.772 0
x[53] 350.834 0
x[54] 353.07 0
x[55] 359.153 0
x[56] 363.153 0
x[57] 370.364 0
x[58] 378.849 0
x[59] 389.29 0
x[60] 397.892 0
x[61] 402.134 0
x[62] 409.206 0
x[63] 415.206 0
x

In [22]:
for v in P_l.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

x[0] 1134.46 0
x[1] 1139.46 0
x[2] 1150.77 0
x[3] 1161.95 0
x[4] 1169.76 0
x[5] 1177.83 0
x[6] 1184.9 0
x[7] 1187.9 0
x[8] 1195.96 0
x[9] 1204.96 0
x[10] 1215.73 0
x[11] 1225.73 0
x[12] 1231.73 0
x[13] 1243.13 0
x[14] 1244.55 0
x[15] 1250.2 0
x[16] 1253.37 0
x[17] 1256.37 0
x[18] 1263.44 0
x[19] 1272.92 0
x[20] 1277.4 0
x[21] 1281.64 0
x[22] 1285.64 0
x[23] 1289.24 0
x[24] 1294.9 0
x[25] 1298.06 0
x[26] 1300.89 0
x[27] 1309.44 0
x[28] 1318.87 0
x[29] 1324.87 0
x[30] 1328.03 0
x[31] 1334.43 0
x[32] 1338.43 0
x[33] 1342.56 0
x[34] 1351.56 0
x[35] 1356.94 0
x[36] 1367.76 0
x[37] 1371.88 0
x[38] 1376.35 0
x[39] 1379.18 0
x[40] 1382.35 0
x[41] 1386.82 0
x[42] 1389.98 0
x[43] 1398.58 0
x[44] 1401.58 0
x[45] 1403.82 0
x[46] 1408.82 0
x[47] 1413.82 0
x[48] 1416.05 0
x[49] 1419.22 0
x[50] 1428.16 0
x[51] 1432.4 0
x[52] 1433.4 0
x[53] 1436.4 0
x[54] 1438.64 0
x[55] 1447.86 0
x[56] 1449.27 0
x[57] 1460.59 0
x[58] 1470.02 0
x[59] 1479.45 0
x[60] 1480.87 0
x[61] 1490.09 0
x[62] 1498.69 0
x[63] 1498

In [ ]:
for constr in P_f.getConstrs()+P_l.getConstrs():
    print(constr.Pi, constr.CBasis)

### 3. 前半後半それぞれのPrimalの最適解を、全体のDualに入れて解く

In [40]:
# 初期解の保存
PStarts = np.array([constr.Pi for constr in P_f.getConstrs()+P_l.getConstrs()])
y3 = np.zeros((Ax.shape[0]-PStarts.shape[0],))
PStarts = np.append(PStarts, y3)

DStarts = np.array([var.X for var in P_f.getVars()+P_l.getVars()])
c3 = np.zeros((PStarts.shape[0],))
DStarts = np.append(DStarts, c3)

In [ ]:
print(f"Ax.T.shape={Ax.T.shape}\t\t\tAp.T.shape={Ap.T.shape}")
for Ax_i, Ap_i in zip(Ax.T, Ap.T):
    print(Ax_i, "\t", Ap_i)

In [61]:
# Gurobiによって最適解を求める
D = solve_dual(Ax, Ap, b, c, instance_name+"D", PStarts.shape[0], PStarts, DStarts)

Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4374 rows, 2624 columns and 7872 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Crossover log...

       0 DPushes remaining with DInf 6.8649177e+05                 0s

     497 PPushes remaining with PInf 1.7273200e+05                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 6.8441198e+05      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    1651   -0.0000000e+00   0.000000e+00   6.844120e+05      0s

Solved in 3123 iterations and 0.25 seconds
Optimal objective  8.913265696e+05


In [62]:
# 最適解
for var in D.getVars():
    print(var.varName, var.X)

y[0] 868.0
y[1] 869.0
y[2] 870.0
y[3] 871.0
y[4] 870.0
y[5] 869.0
y[6] 868.0
y[7] 867.0
y[8] 866.0
y[9] 865.0
y[10] 864.0
y[11] 863.0
y[12] 862.0
y[13] 861.0
y[14] 860.0
y[15] 859.0
y[16] 858.0
y[17] 857.0
y[18] 856.0
y[19] 855.0
y[20] 854.0
y[21] 853.0
y[22] 852.0
y[23] 851.0
y[24] 850.0
y[25] 849.0
y[26] 848.0
y[27] 847.0
y[28] 846.0
y[29] 845.0
y[30] 844.0
y[31] 843.0
y[32] 842.0
y[33] 841.0
y[34] 840.0
y[35] 839.0
y[36] 838.0
y[37] 837.0
y[38] 836.0
y[39] 835.0
y[40] 834.0
y[41] 833.0
y[42] 832.0
y[43] 831.0
y[44] 830.0
y[45] 829.0
y[46] 828.0
y[47] 827.0
y[48] 826.0
y[49] 825.0
y[50] 824.0
y[51] 823.0
y[52] 822.0
y[53] 821.0
y[54] 820.0
y[55] 819.0
y[56] 818.0
y[57] 817.0
y[58] 816.0
y[59] 815.0
y[60] 814.0
y[61] 813.0
y[62] 812.0
y[63] 811.0
y[64] 810.0
y[65] 809.0
y[66] 808.0
y[67] 807.0
y[68] 806.0
y[69] 805.0
y[70] 804.0
y[71] 803.0
y[72] 802.0
y[73] 801.0
y[74] 800.0
y[75] 799.0
y[76] 798.0
y[77] 797.0
y[78] 796.0
y[79] 795.0
y[80] 794.0
y[81] 793.0
y[82] 792.0
y[83] 791.0
y[

y[695] 179.0
y[696] 178.0
y[697] 177.0
y[698] 176.0
y[699] 175.0
y[700] 174.0
y[701] 173.0
y[702] 172.0
y[703] 171.0
y[704] 170.0
y[705] 169.0
y[706] 168.0
y[707] 167.0
y[708] 166.0
y[709] 165.0
y[710] 164.0
y[711] 163.0
y[712] 162.0
y[713] 161.0
y[714] 160.0
y[715] 159.0
y[716] 158.0
y[717] 157.0
y[718] 156.0
y[719] 155.0
y[720] 154.0
y[721] 153.0
y[722] 152.0
y[723] 151.0
y[724] 150.0
y[725] 149.0
y[726] 148.0
y[727] 147.0
y[728] 146.0
y[729] 145.0
y[730] 144.0
y[731] 143.0
y[732] 142.0
y[733] 141.0
y[734] 140.0
y[735] 139.0
y[736] 138.0
y[737] 137.0
y[738] 136.0
y[739] 135.0
y[740] 134.0
y[741] 133.0
y[742] 132.0
y[743] 131.0
y[744] 130.0
y[745] 129.0
y[746] 128.0
y[747] 127.0
y[748] 126.0
y[749] 125.0
y[750] 124.0
y[751] 123.0
y[752] 122.0
y[753] 121.0
y[754] 120.0
y[755] 119.0
y[756] 118.0
y[757] 117.0
y[758] 116.0
y[759] 115.0
y[760] 114.0
y[761] 113.0
y[762] 112.0
y[763] 111.0
y[764] 110.0
y[765] 109.0
y[766] 108.0
y[767] 107.0
y[768] 106.0
y[769] 105.0
y[770] 104.0
y[771] 103.0

y[1945] 1.0
y[1946] 0.0
y[1947] 1.0
y[1948] 0.0
y[1949] 1.0
y[1950] 0.0
y[1951] 1.0
y[1952] 0.0
y[1953] 1.0
y[1954] 0.0
y[1955] 1.0
y[1956] 0.0
y[1957] 1.0
y[1958] 0.0
y[1959] 1.0
y[1960] 0.0
y[1961] 1.0
y[1962] 0.0
y[1963] 1.0
y[1964] 0.0
y[1965] 1.0
y[1966] 0.0
y[1967] 1.0
y[1968] 0.0
y[1969] 1.0
y[1970] 0.0
y[1971] 1.0
y[1972] 0.0
y[1973] 1.0
y[1974] 0.0
y[1975] 1.0
y[1976] 0.0
y[1977] 1.0
y[1978] 0.0
y[1979] 1.0
y[1980] 0.0
y[1981] 1.0
y[1982] 0.0
y[1983] 1.0
y[1984] 0.0
y[1985] 1.0
y[1986] 0.0
y[1987] 1.0
y[1988] 0.0
y[1989] 1.0
y[1990] 0.0
y[1991] 1.0
y[1992] 0.0
y[1993] 1.0
y[1994] 0.0
y[1995] 1.0
y[1996] 0.0
y[1997] 1.0
y[1998] 0.0
y[1999] 1.0
y[2000] 0.0
y[2001] 1.0
y[2002] 0.0
y[2003] 1.0
y[2004] 0.0
y[2005] 1.0
y[2006] 0.0
y[2007] 1.0
y[2008] 0.0
y[2009] 1.0
y[2010] 0.0
y[2011] 1.0
y[2012] 0.0
y[2013] 1.0
y[2014] 0.0
y[2015] 1.0
y[2016] 0.0
y[2017] 1.0
y[2018] 0.0
y[2019] 1.0
y[2020] 0.0
y[2021] 1.0
y[2022] 0.0
y[2023] 1.0
y[2024] 0.0
y[2025] 1.0
y[2026] 0.0
y[2027] 1.0
y[20

### 4. 全体のPrimalの最適解を得る

In [63]:
for i, constr in enumerate(D.getConstrs()):
    print(constr.Pi)

0.0
-2.23606797749979
-4.47213595499958
-7.63441361516796
-17.533908551779625
-28.164054364514275
-39.47776286349904
-50.87951711449042
-59.42352085980795
-63.02907213527194
-71.0913298835705
-77.79953381606987
-88.61618764246184
-94.44713953730714
-98.44713953730714
-100.68320751480692
-102.91927549230671
-108.91927549230671
-111.91927549230671
-116.3914114473063
-117.80562500967939
-119.80562500967939
-126.13018033001615
-130.2532859556338
-137.5333958449143
-145.34364552082096
-156.1602993472129
-164.22255709551146
-172.0328067714181
-180.5180881456567
-186.60085067595492
-196.8964808169419
-204.1075833678699
-211.31868591879788
-213.55475389629768
-214.9689674586708
-220.06798697226358
-223.23026463243195
-233.52589477341894
-244.8396032724037
-256.881197851196
-265.82546976119517
-275.25945089325177
-285.88959670598643
-289.05187436615483
-295.13463689645306
-302.94488657235974
-306.55043784782373
-311.55043784782373
-322.9521920988151
-329.66039603131446
-336.0635202687473
-342.7

-1662.5665747037428
-1660.1721259792066
-1665.2431937910724
-1664.4054714512404
-1666.11367538374
-1661.5278889461133
-1660.5278889461133
-1666.3381386220199
-1663.57420659952
-1667.898761919857
-1671.1788718091375
-1672.0072989338832
-1677.106318447476
-1678.268596107644
-1681.268596107644
-1681.3676156212368
-1682.0244698707293
-1681.0244698707293
-1679.8554217655746
-1678.0914897430748
-1684.635493488392
-1688.8335325155776
-1695.0138724030767
-1699.6161976701196
-1699.739303295737
-1702.739303295737
-1703.739303295737
-1699.9753712732372
-1700.0984768988546
-1699.0984768988546
-1703.0984768988546
-1699.2607545590226
-1696.674968121396
-1698.7577306516941
-1702.8404931819923
-1704.9635988076097
-1705.5691500830735
-1701.8052180605737
-1708.6154677364802
-1712.8350121937729
-1717.0812234450086
-1717.4057787653455
-1723.4885412956437
-1720.7246092731439
-1721.5555611679893
-1721.5555611679893
-1726.7666637189168
-1724.7666637189168
-1728.7666637189168
-1733.828921467215
-1737.88430660

In [57]:
for var in P.getVars():
    print(var.varName, var.X)

x[0] 0.0
x[1] 2.23606797749979
x[2] 4.47213595499958
x[3] 7.63441361516796
x[4] 17.533908551779625
x[5] 28.164054364514275
x[6] 39.47776286349904
x[7] 50.87951711449042
x[8] 59.42352085980795
x[9] 63.02907213527194
x[10] 71.0913298835705
x[11] 77.79953381606987
x[12] 88.61618764246184
x[13] 94.44713953730714
x[14] 98.44713953730714
x[15] 100.68320751480692
x[16] 102.91927549230671
x[17] 108.91927549230671
x[18] 111.91927549230671
x[19] 116.3914114473063
x[20] 117.80562500967939
x[21] 119.80562500967939
x[22] 126.13018033001615
x[23] 130.2532859556338
x[24] 137.5333958449143
x[25] 145.34364552082096
x[26] 156.1602993472129
x[27] 164.22255709551146
x[28] 172.0328067714181
x[29] 180.5180881456567
x[30] 186.60085067595492
x[31] 196.8964808169419
x[32] 204.1075833678699
x[33] 211.31868591879788
x[34] 213.55475389629768
x[35] 214.9689674586708
x[36] 220.06798697226358
x[37] 223.23026463243195
x[38] 233.52589477341894
x[39] 244.8396032724037
x[40] 256.881197851196
x[41] 265.82546976119517
x[4

x[757] 4299.550435003426
x[758] 4299.550435003426
x[759] 4304.022570958426
x[760] 4312.566574703743
x[761] 4316.172125979207
x[762] 4323.243193791072
x[763] 4326.40547145124
x[764] 4333.11367538374
x[765] 4334.527888946113
x[766] 4338.527888946113
x[767] 4346.33813862202
x[768] 4348.57420659952
x[769] 4354.898761919857
x[770] 4362.1788718091375
x[771] 4365.007298933883
x[772] 4370.106318447476
x[773] 4373.268596107644
x[774] 4380.268596107644
x[775] 4385.367615621237
x[776] 4391.024469870729
x[777] 4393.024469870729
x[778] 4398.855421765575
x[779] 4401.091489743075
x[780] 4409.635493488392
x[781] 4419.833532515578
x[782] 4431.013872403077
x[783] 4439.61619767012
x[784] 4443.739303295737
x[785] 4448.739303295737
x[786] 4450.739303295737
x[787] 4452.975371273237
x[788] 4457.098476898855
x[789] 4459.098476898855
x[790] 4465.098476898855
x[791] 4468.260754559023
x[792] 4469.674968121396
x[793] 4475.757730651694
x[794] 4481.840493181992
x[795] 4485.96359880761
x[796] 4489.5691500830735
x[79

p[540] 1312.572168338368
p[541] 1313.671187851961
p[542] 1314.9957431722978
p[543] 1315.3809079794323
p[544] 1314.7951215418052
p[545] 1317.7951215418052
p[546] 1317.918227167423
p[547] 1316.1542951449228
p[548] 1317.1542951449228
p[549] 1320.3165728050913
p[550] 1320.3165728050913
p[551] 1320.7017376122258
p[552] 1318.7017376122258
p[553] 1322.7571227503631
p[554] 1320.7571227503631
p[555] 1320.362674025827
p[556] 1324.9784471316907
p[557] 1327.6353013811831
p[558] 1326.7975790413516
p[559] 1328.8803415716498
p[560] 1328.9793610852425
p[561] 1328.810312980088
p[562] 1333.029857437381
p[563] 1328.029857437381
p[564] 1330.1288769509738
p[565] 1330.3399795019018
p[566] 1326.3399795019018
p[567] 1327.9455307773655
p[568] 1323.9455307773655
p[569] 1319.9455307773655
p[570] 1318.9455307773655
p[571] 1314.3597443397384
p[572] 1314.5958123172381
p[573] 1317.5958123172381
p[574] 1319.8759222065187
p[575] 1320.8759222065187
p[576] 1323.8759222065187
p[577] 1321.8759222065187
p[578] 1325.8759222